In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.insert(0, '../src')

from options_toolkit.bsm import bsm_call_price, bsm_put_price, implied_volatility
from options_toolkit.viz import payoff_heatmap
from ipywidgets import interact, FloatSlider, Dropdown

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Implied Vol toolkit loaded! Interactive widgets ready.")


✅ Implied Vol toolkit loaded! Interactive widgets ready.


## 1. Option Price vs Volatility

First, let's see how call option price varies with volatility σ.

**Key insight**: Option prices are monotonically increasing in volatility. Higher volatility means more uncertainty, which increases option value (both calls and puts).


In [ ]:
# 🎛️ INTERACTIVE: Option Price vs Volatility (Call & Put)

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def plot_price_vs_vol(S0=100, K=100, r=0.05, T=1.0):
    """Interactive plot of option price vs volatility for both calls and puts.
    
    This shows the fundamental relationship: option prices are monotonically
    increasing in volatility. Higher uncertainty = higher option value.
    """
    sigma_range = np.linspace(0.05, 0.60, 100)
    call_prices = [bsm_call_price(S0, K, r, sigma, T) for sigma in sigma_range]
    put_prices = [bsm_put_price(S0, K, r, sigma, T) for sigma in sigma_range]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(sigma_range * 100, call_prices, 'b-', linewidth=2.5, label='Call')
    ax.plot(sigma_range * 100, put_prices, 'r--', linewidth=2.5, label='Put')
    ax.axvline(x=20, color='gray', linestyle='--', linewidth=1.5, alpha=0.5, label='σ = 20%')
    ax.set_xlabel('Volatility (%)', fontsize=12)
    ax.set_ylabel('Option Price ($)', fontsize=12)
    ax.set_title(f'Option Price vs Volatility (S=${S0}, K=${K}, T={T:.2f}yr)', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=11)
    plt.tight_layout()
    plt.show()
    
    print("═" * 70)
    print("📊 PRICE SENSITIVITY TO VOLATILITY")
    print("═" * 70)
    print(f"Parameters: S=${S0}, K=${K}, r={r*100:.1f}%, T={T:.2f} years")
    print("-" * 70)
    print(f"Call Prices:")
    print(f"  At σ=10%: ${bsm_call_price(S0, K, r, 0.10, T):.4f}")
    print(f"  At σ=20%: ${bsm_call_price(S0, K, r, 0.20, T):.4f}")
    print(f"  At σ=40%: ${bsm_call_price(S0, K, r, 0.40, T):.4f}")
    print(f"\nPut Prices:")
    print(f"  At σ=10%: ${bsm_put_price(S0, K, r, 0.10, T):.4f}")
    print(f"  At σ=20%: ${bsm_put_price(S0, K, r, 0.20, T):.4f}")
    print(f"  At σ=40%: ${bsm_put_price(S0, K, r, 0.40, T):.4f}")
    print("-" * 70)
    print("Key Insight:")
    print("- Both calls and puts gain value as volatility increases")
    print("- Higher σ → more uncertainty → higher probability of large moves")
    print("- This relationship is why traders pay attention to IV levels")
    print("═" * 70)


interactive(children=(FloatSlider(value=100.0, description='Stock ($):', max=120.0, min=80.0, step=5.0), Float…

In [ ]:
# 🎛️ INTERACTIVE IMPLIED VOLATILITY SOLVER

@interact(
    market_price=FloatSlider(value=10, min=1, max=50, step=0.5, description='Market Price ($):'),
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    K=FloatSlider(value=100, min=80, max=120, step=5, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):'),
    option_type=Dropdown(options=['call', 'put'], value='call', description='Option:')
)
def solve_iv(market_price=10, S0=100, K=100, r=0.05, T=1.0, option_type='call'):
    """Interactive IV solver using Newton-Raphson method.
    
    Implied volatility is the volatility parameter that, when plugged into
    the BSM formula, produces the observed market price. It represents the
    market's expectation of future volatility.
    
    The solver uses Newton-Raphson iteration to find the root of:
        f(σ) = BSM_Price(σ) - Market_Price = 0
    """
    try:
        iv = implied_volatility(market_price, S0, K, r, T, option_type=option_type)
        
        # Verify by pricing with recovered IV
        if option_type == 'call':
            check_price = bsm_call_price(S0, K, r, iv, T)
            intrinsic = max(S0 - K, 0)
        else:
            check_price = bsm_put_price(S0, K, r, iv, T)
            intrinsic = max(K - S0, 0)
        
        print("═" * 70)
        print(f"📊 IMPLIED VOLATILITY SOLUTION")
        print("═" * 70)
        print(f"Parameters: S=${S0}, K=${K}, r={r*100:.1f}%, T={T:.2f} years")
        print(f"Option Type: {option_type.upper()}")
        print("-" * 70)
        print(f"Market Price:      ${market_price:.4f}")
        print(f"Intrinsic Value:   ${intrinsic:.4f}")
        print(f"Time Value:        ${market_price - intrinsic:.4f}")
        print("-" * 70)
        print(f"Implied Vol (IV):  {iv*100:.2f}%")
        print(f"Verification:      ${check_price:.4f} (BSM price with IV)")
        print(f"Error:             ${abs(check_price - market_price):.8f}")
        print("═" * 70)
        print(f"\n✅ The market is pricing this {option_type} at {iv*100:.2f}% annualized volatility")
        print(f"\n💡 Interpretation:")
        print(f"   - IV represents the market's expectation of volatility")
        print(f"   - High IV → market expects large price swings")
        print(f"   - Low IV → market expects stable prices")
        print(f"   - IV can differ from historical volatility (realized vol)")
        print("═" * 70)
        
    except ValueError as e:
        print(f"❌ Error: {e}")
        print(f"\n💡 Tip: Market price must be > intrinsic value (min option value)")
        print(f"   For a call: price > max(S-K, 0)")
        print(f"   For a put:  price > max(K-S, 0)")


interactive(children=(FloatSlider(value=10.0, description='Market Price ($):', max=50.0, min=1.0, step=0.5), F…

In [ ]:
## 3. IV Environment Comparison: Low vs High Volatility

Compare option prices in low IV vs high IV environments. This demonstrates why
traders monitor IV levels—high IV means expensive options.


 Strike  Call Price (IV=15%)  Call Price (IV=35%)  Difference
     90            15.467159            21.321119    5.853960
     95            11.753845            18.583271    6.829426
    100             8.591658            16.128429    7.536771
    105             6.035564            13.944175    7.908612
    110             4.075866            12.014090    7.938224

Observation: High IV environment has much higher option premiums!
ATM call: $8.59 (low IV) vs $16.13 (high IV)


In [ ]:
# 🎛️ INTERACTIVE: Compare Low vs High IV Environment

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):'),
    low_iv=FloatSlider(value=0.15, min=0.05, max=0.30, step=0.05, description='Low IV (%):'),
    high_iv=FloatSlider(value=0.35, min=0.20, max=0.60, step=0.05, description='High IV (%):')
)
def compare_iv_environments(S0=100, r=0.05, T=1.0, low_iv=0.15, high_iv=0.35):
    """Compare option prices in low vs high IV environments."""
    strikes = np.array([S0*0.9, S0*0.95, S0, S0*1.05, S0*1.1])
    
    low_iv_prices_call = [bsm_call_price(S0, K_val, r, low_iv, T) for K_val in strikes]
    high_iv_prices_call = [bsm_call_price(S0, K_val, r, high_iv, T) for K_val in strikes]
    low_iv_prices_put = [bsm_put_price(S0, K_val, r, low_iv, T) for K_val in strikes]
    high_iv_prices_put = [bsm_put_price(S0, K_val, r, high_iv, T) for K_val in strikes]
    
    # Create comparison table
    df = pd.DataFrame({
        'Strike': strikes.round(2),
        f'Call (IV={low_iv*100:.0f}%)': [f'${p:.2f}' for p in low_iv_prices_call],
        f'Call (IV={high_iv*100:.0f}%)': [f'${p:.2f}' for p in high_iv_prices_call],
        f'Put (IV={low_iv*100:.0f}%)': [f'${p:.2f}' for p in low_iv_prices_put],
        f'Put (IV={high_iv*100:.0f}%)': [f'${p:.2f}' for p in high_iv_prices_put],
    })
    
    print("═" * 80)
    print("📊 OPTION PRICES: LOW IV vs HIGH IV ENVIRONMENT")
    print("═" * 80)
    print(f"Parameters: S=${S0}, r={r*100:.1f}%, T={T:.2f} years")
    print("-" * 80)
    print(df.to_string(index=False))
    print("-" * 80)
    
    atm_call_low = low_iv_prices_call[2]  # ATM (strike = S0)
    atm_call_high = high_iv_prices_call[2]
    premium_increase = ((atm_call_high - atm_call_low) / atm_call_low) * 100
    
    print(f"\nATM Call Premium Increase:")
    print(f"  Low IV ({low_iv*100:.0f}%):  ${atm_call_low:.2f}")
    print(f"  High IV ({high_iv*100:.0f}%): ${atm_call_high:.2f}")
    print(f"  Increase: {premium_increase:.1f}%")
    print("═" * 80)
    print("\n💡 Key Insight:")
    print("   - High IV environments → Much more expensive options")
    print("   - Traders prefer selling options when IV is high (collect more premium)")
    print("   - Traders prefer buying options when IV is low (cheaper entry)")
    print("   - IV mean reversion: High IV tends to decline (IV crush)")
    print("═" * 80)


## 4. Heatmap: Call Price vs Stock Price and Volatility

Let's create a 2D heatmap showing how call option price varies with both underlying price and volatility.


In [ ]:
# 🎛️ INTERACTIVE VOLATILITY SURFACE HEATMAP

@interact(
    S0=FloatSlider(value=100, min=80, max=120, step=5, description='Stock ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    base_vol=FloatSlider(value=0.20, min=0.10, max=0.40, step=0.05, description='Base Vol (%):')
)
def plot_vol_surface(S0=100, r=0.05, base_vol=0.20):
    """Interactive volatility surface visualization.
    
    This shows option prices across strikes and maturities. In real markets,
    IV often forms a 'smile' (U-shaped) or 'skew' (sloped) pattern across strikes.
    """
    strikes = np.linspace(S0*0.8, S0*1.2, 40)
    T_vals = np.linspace(0.25, 2.0, 30)
    
    # Create a stylized volatility surface (simulating IV smile/skew)
    # In practice, this would come from market data
    prices = np.zeros((len(T_vals), len(strikes)))
    
    for i, T in enumerate(T_vals):
        for j, K in enumerate(strikes):
            # Simple stylized smile: IV increases as strike moves away from S0
            # This creates a U-shaped pattern (volatility smile)
            moneyness = K / S0
            vol_adjustment = base_vol * (1 + 0.3 * (moneyness - 1.0)**2)
            prices[i, j] = bsm_call_price(S0, K, r, vol_adjustment, T)
    
    fig = payoff_heatmap(strikes, T_vals, prices,
                         xlabel="Strike Price ($)",
                         ylabel="Time to Expiration (years)",
                         title=f"Call Option Price Surface (S=${S0}, r={r*100:.1f}%)",
                         cmap="viridis")
    plt.show()
    
    print("═" * 70)
    print("📊 IMPLIED VOLATILITY SURFACE INTERPRETATION")
    print("═" * 70)
    print("This heatmap shows option prices across strikes and maturities.")
    print("-" * 70)
    print("Common Patterns in Real Markets:")
    print("1. **Volatility Smile** (U-shaped): IV highest for OTM calls and OTM puts")
    print("   → Market assigns higher probability to extreme moves")
    print("2. **Volatility Skew** (sloped): IV higher for OTM puts than OTM calls")
    print("   → 'Fear' premium (investors pay more for downside protection)")
    print("3. **Term Structure**: IV often varies by maturity")
    print("   → Short-term events (earnings) vs long-term uncertainty")
    print("-" * 70)
    print("Trading Implications:")
    print("- Compare current IV to historical IV levels")
    print("- Watch for IV expansion (increasing) or IV crush (decreasing)")
    print("- Sell options when IV is high, buy when IV is low")
    print("═" * 70)


In [11]:
# 🎛️ INTERACTIVE VOLATILITY SURFACE HEATMAP

@interact(
    K=FloatSlider(value=100, min=80, max=120, step=10, description='Strike ($):'),
    r=FloatSlider(value=0.05, min=0.0, max=0.15, step=0.01, description='Rate:'),
    T=FloatSlider(value=1.0, min=0.25, max=2.0, step=0.25, description='Time (yrs):')
)
def plot_vol_surface(K=100, r=0.05, T=1.0):
    """Interactive volatility surface."""
    S_vals = np.linspace(K*0.7, K*1.3, 50)
    sigma_vals = np.linspace(0.10, 0.50, 40)
    
    prices = np.zeros((len(sigma_vals), len(S_vals)))
    for i, sigma in enumerate(sigma_vals):
        for j, S in enumerate(S_vals):
            prices[i, j] = bsm_call_price(S, K, r, sigma, T)
    
    fig = payoff_heatmap(S_vals, sigma_vals, prices,
                         xlabel="Stock Price ($)",
                         ylabel="Volatility (σ)",
                         title=f"Call Option Price Surface (K=${K}, T={T}yr)",
                         cmap="viridis")
    plt.show()
    
    print(f"📊 Heatmap shows option value across stock prices and volatilities")
    print(f"   Bright regions = High option value | Dark regions = Low value")


interactive(children=(FloatSlider(value=100.0, description='Strike ($):', max=120.0, min=80.0, step=10.0), Flo…

## Summary

In this notebook, we:
1. ✅ Explored the relationship between volatility and option prices
2. ✅ Solved for implied volatility from market prices (Newton-Raphson method)
3. ✅ Compared low IV vs high IV environments
4. ✅ Visualized volatility surfaces (IV across strikes and maturities)

### Key Takeaways:
- **Implied Volatility (IV)** represents the market's expectation of future volatility
- **IV is "forward-looking"** (market expectation) vs historical volatility (past realized)
- **High IV = Expensive options**, Low IV = Cheap options
- **IV patterns** (smile/skew) reveal market sentiment and risk perceptions
- **IV mean reversion** → High IV often declines (IV crush after events)
- **IV vs Realized Vol** → Compare to identify mispricing opportunities

### Next Steps:
- Return to **Notebook 1** to see how Greeks vary with volatility
- Explore **Monte Carlo pricing** and **strategies** in Notebook 3
